In [ ]:
###### This is a sample Python script.

from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.models import Model
import numpy as np
from numpy import loadtxt
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from imblearn.over_sampling import SMOTE
from sklearn import metrics
import os
import sys
from tensorflow.keras.models import load_model
from pandas import read_csv
from numpy import genfromtxt
debug = True


f1_scores = []
fpr = []
tpr = []
            
print('Original Dataset')
PCA = genfromtxt("/kaggle/input/amazon-kaggle-with-pca/Amazon_Kaggle_with_PCA.csv", delimiter=',')
ids1 = PCA[:,:2]
meta1 = PCA[:,2:-1]
output1 = amazon[:,-1:]
print("done")
print(ids1)
print(meta1.shape)
meta1 = meta1[:32769]
print(meta1.shape)
print(output1.shape)
x_train, x_test, y_train, y_test = train_test_split(meta1 ,output1,test_size = 0.2, random_state=1)



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

def create_tabular_vgg16(input_shape):
    model = Sequential()

    # Flatten the input
    model.add(Flatten(input_shape=input_shape))

    # Fully connected layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Adjust output layer based on your task (e.g., binary classification)

    return model

# Define input shape based on the number of features in your tabular data
input_shape = (9,)  # Replace num_features with the actual number of features in your dataset

# Create tabular VGG16 model
model = create_tabular_vgg16(input_shape)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 60 epochs seems to be the sweet spot; tried 10, 20, 30, and 100
# batch size seems better at 16; tried 8, 32, 64 
model.fit(x_train, y_train, epochs=60, batch_size=16, validation_data=(x_test, y_test))



In [ ]:
y_pred = model.predict(x_test)
score = model.evaluate(x_test, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1]) 

outputFileName = 'vgg16_amazon_kaggle_pca'
DIR_ASSETS = 'results/'
PATH_MODEL = DIR_ASSETS + outputFileName + '.hdf5'

if debug:
  print('Saving trained vgg16 to {}.'.format(PATH_MODEL))
if not os.path.isdir(DIR_ASSETS):
    os.mkdir(DIR_ASSETS)
model.save(PATH_MODEL)

# measure True Positive/ Negative, False Positive/ Negative

from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, confusion_matrix, f1_score
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn import metrics

y_pred = model.predict(x_test)

# Flatten the predicted and actual values
y_pred_flat = y_pred.flatten()
y_test_flat = y_test.flatten()

# Convert to binary classification (example)
# Adjust this step according to your specific problem
threshold = 0.5
y_pred_binary = (y_pred_flat > threshold).astype(int)

# Reshape y_test_flat to match the shape of y_pred_flat
y_test_binary = (y_test_flat[:len(y_pred_binary)] > threshold).astype(int)
#y_pred = to_categorical(y_pred)
f1 = f1_score(y_test_binary, y_pred_binary, average='weighted')
# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)

cm = confusion_matrix(y_test_binary.ravel(), y_test_binary.ravel())
#np.savetxt('cm.txt', cm, delimiter=',', fmt='%f')
#f1 = f1_score(y_test.ravel(), y_pred.ravel())
tpr_value = cm[1, 1] / (cm[1, 1] + cm[1, 0])
fpr_value = cm[0, 1] / (cm[0, 1] + cm[0, 0])

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)
print("TPR Score:", tpr_value)
print("FPR Score:", fpr_value)


In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

precision_a, recall_a, threshold = precision_recall_curve(y_test_binary, y_pred_binary)
prc_auc = auc(recall_a, precision_a)
print("Area Under the PR Curve score: ", prc_auc)


fpr, tpr, threshold = roc_curve(y_test_binary, y_pred_binary)
roc_auc = auc(fpr, tpr)
print("Area Under the ROC Curve score: ", roc_auc)

In [ ]:
import json

# Assuming tpr and fpr are NumPy arrays
print("Type of f1:", type(f1))
print("Type of precision:", type(precision))
print("Type of recall:", type(recall))
print("Type of tpr_value:", type(tpr_value))
print("Type of fpr_value:", type(fpr_value))
print("Type of roc_auc:", type(roc_auc))
print("Type of prc_auc:", type(prc_auc))
print("Type of tpr:", type(tpr))
print("Type of fpr:", type(fpr))

precision_list = precision_a.tolist()
recall_list = recall_a.tolist()
tpr_list = tpr.tolist()
fpr_list = fpr.tolist()

#print(f1.type())

data = {
    "Average F1 Score": f1, 
    "Average Precision": precision, 
    "Average Recall": recall, 
    "Average True Positive Rate": tpr_value, 
    "Average False Positive Rate": fpr_value, 
    "Average ROC AUC": roc_auc, 
    "Average PRC AUC": prc_auc,
    "TPR Array" : tpr_list,
    "FPR Array" : fpr_list,
    "Precision Array": precision_list, 
    "Recall Array": recall_list
}

# Specify the file path
file_path = "/kaggle/working/VGG-16 - Amazon Kaggle with PCA.json"

# Write the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)